In [ ]:
    '''
    --Step 4: Gather past metrics and create the basic heuristic forecast, plus median metrics tables
    create or replace table max_dev.workspace.psi_past_base_assets_temp as 
    (select distinct
          a.title_id
        , coalesce(a.season_number,0) as season_number
        , a.viewable_id
        , title_name
        , first_offered_date::date as asset_max_premiere
        , end_utc_max::date as asset_max_end_dt
        , coalesce(raod.season_first_offered_date::date,raod.title_first_offered_date::date) as season_premiere
        , asset_run_time
        , a.content_category
        , episode_number_in_season
        , content_source
        , program_type
        , category
        , tier
        , viewership_start_date as effective_start_date
        , viewership_end_date as effective_end_date
    from max_prod.catalog.reporting_asset_dim a
    join max_prod.catalog.reporting_asset_offering_dim raod
        on a.viewable_id = raod.viewable_id
        and brand = 'HBO MAX'
        and territory = 'HBO MAX DOMESTIC'
        and channel = 'HBO MAX SUBSCRIPTION'
    inner join max_prod.content_analytics.psi_past_title_metadata b
        on a.title_id = b.viewership_title_id
        and coalesce(a.season_number,0) = coalesce(b.viewership_season_number,0)
    where 1 = 1
    and asset_type IN ('FEATURE','ELEMENT')
    and start_utc_max is not null
    and a.content_category in ('movies','series','special')
    and coalesce(raod.season_first_offered_date,raod.title_first_offered_date)  >= '2020-05-27 07:01:00.000'
    order by season_premiere, title_name 
    );
    select * from max_dev.workspace.psi_past_base_assets_temp;
    '''
    
    '''
    with fv as (
    select
          b.title_id
        , b.title_name
        , b.season_number
        , b.content_category
        , b.category
        , tier
        , request_time_gmt::date as request_date
        , count(distinct concat(hbo_uuid, subscription_id)) as first_views
    from MAX_PROD.BI_ANALYTICS.SUBSCRIPTION_FIRST_CONTENT_WATCHED a
    inner join max_dev.workspace.psi_past_base_assets_temp b
        on a.viewable_id = b.viewable_id
        --and request_time_gmt::date between season_premiere_date and dateadd('day',90,season_premiere_date)
        --and season_premiere_date >= '2020-05-27 07:00:01'
    where 1 = 1
        and request_time_gmt::date between asset_max_premiere and asset_max_end_dt
        and request_time_gmt::date between effective_start_date and effective_end_date
        and request_time_gmt::date < dateadd('days',-1,$val_date)
        and country_iso_code in ('US','PR','GU')
    group by 1,2,3,4,5,6,7
    --order by 2,4
    '''